In [278]:
import multiprocessing
import pycurl
import pymysql
import urllib.parse
import io
import re
import json
import requests
from lxml.html import fromstring, tostring

In [294]:
con = pymysql.connect(host='localhost', user='root', passwd='', db='xiaobaifinder', charset='utf8')
cu_r = con.cursor()
cu_r.execute('select pmid,link from doi_link where link like "%acs.org%" limit 100,10')
link_list = cu_r.fetchall()
print(link_list)
con.close()

(('26647786', 'http://pubs.acs.org/doi/10.1021/acsami.5b09094'), ('26647284', 'http://pubs.acs.org/doi/10.1021/acsami.5b10786'), ('26647225', 'http://pubs.acs.org/doi/10.1021/acsnano.5b06716'), ('26647019', 'http://pubs.acs.org/doi/10.1021/acsnano.5b05596'), ('26647005', 'http://pubs.acs.org/doi/10.1021/acs.molpharmaceut.5b00630'), ('26646791', 'http://pubs.acs.org/doi/10.1021/acsnano.5b06011'), ('26646753', 'http://pubs.acs.org/doi/10.1021/acs.bioconjchem.5b00569'), ('26646667', 'http://pubs.acs.org/doi/10.1021/acs.orglett.5b03192'), ('26646584', 'http://pubs.acs.org/doi/10.1021/acsnano.5b05406'), ('26646554', 'http://pubs.acs.org/doi/10.1021/acs.orglett.5b03223'))


In [350]:
url = 'http://pubs.acs.org/doi/10.1021/acsami.5b09094'

In [351]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36'}

text = requests.get(url, headers = headers).content.decode('utf-8')

In [352]:
acs(11, text)

11 1 Wushuang Bai Institute of Analytical Science, Shaanxi Provincial KeyLaboratory of Electroanalytical Chemistry, Northwest University, Xi’an, Shaanxi 710069, China None 0
11 2 Qinglin Sheng Institute of Analytical Science, Shaanxi Provincial KeyLaboratory of Electroanalytical Chemistry, Northwest University, Xi’an, Shaanxi 710069, China None 0
11 3 Fei Nie Institute of Analytical Science, Shaanxi Provincial KeyLaboratory of Electroanalytical Chemistry, Northwest University, Xi’an, Shaanxi 710069, China None 0
11 4 Jianbin Zheng Institute of Analytical Science, Shaanxi Provincial KeyLaboratory of Electroanalytical Chemistry, Northwest University, Xi’an, Shaanxi 710069, China zhengjb@nwu.edu.cn 1
['controllable synthesis', 'liquid-gas reaction', 'manganese oxide', 'electrochemical sensing', 'hydrogen peroxide']


In [348]:
def acs(pmid, text):
    root = fromstring(text)
    # aff
    aff_dict={}
    aff_index = 0
    aff_list = root.cssselect('div.affiliations div')
    first = None
    if len(aff_list)==1:
        first=1
    for aff in aff_list:
        aff_index +=1
        
        if not first:
            sup = aff.find('sup').text_content()
#             print(sup)
            aff_name = aff.text_content().replace(sup, '').replace('\r\n', '').strip()
            aff_dict[sup] = aff_name
#             print(aff_name)
        else:
            aff_name = aff.text_content().replace('\r\n', '').strip()
            aff_dict['first']=aff_name
#         cu.execute('insert into journal_aff (pmid, aff_index, aff_name) values(%s, %s, %s)', (pmid, aff_index, aff_name,))
#         con.commit()
    corresp_email = [email.text for email in root.cssselect('div#correspondence a')]
    # author
    author_index=0

    equal = re.compile('Authors with equal contribution').findall(text)
    
    for author_li in root.cssselect('div#authors>span.hlFld-ContribAuthor'):
#         print(tostring(author_li))
        corresp=0
        author=None
        email = None
        author_aff_name = None
        author_index +=1
        author_aff_list=[]
        author = author_li.text_content().replace(',', '').replace('and', '').replace('*', '').strip()
        author = re.compile('[0-9]').sub('', author)
        if not first:
            sups = author_li.findall('a.NLM_xref-aff')
            for sup in sups:
                aff_name = aff_dict[sup.text]
            author_aff_list.append(aff_name)
        else:
            author_aff_list.append(aff_dict['first'])
        authro_corresp = author_li.cssselect('a.ref')
#         print(tostring(authro_corresp[0]).text_content)
        if authro_corresp and authro_corresp[0].text_content() == '*':
            
            corresp=1
            email = corresp_email.pop(0)
        for author_aff_name in author_aff_list:
#             pass
            print(pmid, author_index, author, author_aff_name, email, corresp)
#             cu.execute('insert into journal_author values(%s, %s, %s, %s, %s, %s)',
#                (pmid, author_index, author, author_aff_name, email, corresp,))
#             con.commit() 
    # keywords
    keywords = [kw.text for kw in  root.cssselect('div.keywords a')]
    print(keywords)

In [293]:
con = pymysql.connect(host='localhost', user='root', passwd='', db='xiaobaifinder', charset='utf8')
cu = con.cursor()
# cu.execute('create table if not exists journal_keyword(pmid int, keywords varchar(200))')
# cu.execute('create table if not exists journal_aff(pmid int,aff_index int, aff_name Text)')
# cu.execute('create table if not exists journal_author(pmid int,author_index int,\
#             author varchar(200), author_aff Text, email varchar(200), corresp int)')
# con.commit()
pmid=1
# text = elsevier(pmid, url)
wiley(11, url)
cu.close()
con.close()

[<Element a at 0x107d009f8>]
b'<li id="adma201504747-cr-0001">Liubing Dong<sup>1,2</sup>, </li>'
['1', '2']
b'<li id="adma201504747-cr-0002">Chengjun Xu<sup>1,*</sup>, </li>'
['1', '*']
vivaxuchengjun@163.com []
b'<li id="adma201504747-cr-0003">Yang Li<sup>3</sup>, </li>'
['3']
b'<li id="adma201504747-cr-0004">Changle Wu<sup>1</sup>, </li>'
['1']
b'<li id="adma201504747-cr-0005">Baozheng Jiang<sup>1</sup>, </li>'
['1']
b'<li id="adma201504747-cr-0006">Qian Yang<sup>1</sup>, </li>'
['1']
b'<li id="adma201504747-cr-0007">Enlou Zhou<sup>4</sup>, </li>'
['4']
b'<li id="adma201504747-cr-0008">Feiyu Kang<sup>1,2,*</sup> and</li>'
['1', '2', '*']
fykang@mail.tsinghua.edu.cn []
b'<li id="adma201504747-cr-0009">Quan-Hong Yang<sup>1</sup></li>'
['1']


In [292]:
def wiley(pmid, url):
    # wiley 存在问题， 个别时候 * 即表示共同第一又表示通讯, 已解决
    text = requests.get(url, headers = headers).content
    root = fromstring(text)
    new = root.cssselect('a#wol1backlink')
    if new:
        print(new)
        url= new[0].get('href')
        text = requests.get(url, headers = headers).content
        root = fromstring(text)
    # aff
    #authorsAffiliations
    if not isinstance(text, str):
        text = text.decode('utf-8')
    aff_dict={}
    aff_index = 0
    aff_list = root.cssselect('ol#authorsAffiliations li')
    first = None # 单位的数目
    if len(aff_list)==1:
        first=1
    for aff in aff_list:
        aff_index +=1
        aff_name = aff.find('p').text_content()
        if not first:
            sup = aff.find('span').text_content()
            aff_dict[sup] = aff_name
        else:
            aff_dict['first']=aff_name
        cu.execute('insert into journal_aff (pmid, aff_index, aff_name) values(%s, %s, %s)', (pmid, aff_index, aff_name,))
        con.commit()
    corresp_email = [email.text for email in root.cssselect('p#correspondence a')]
    # author
    author_index=0

    equal = re.compile('Authors with equal contribution').findall(text)
    
    for author_li in root.cssselect('ol#authors li'):
        print(tostring(author_li))
        corresp=0
        author=None
        email = None
        author_aff_name = None
        author_index +=1
        author = author_li.text_content().replace(',', '').replace('and', '').replace('*', '').strip()
        author = re.compile('[0-9]').sub('', author)
        author_aff_list = []
        author_sup = author_li.find('sup')
        if author_sup is not None: 
            sups = author_sup.text_content().split(',')
            print(sups)
            for sup in sups:
                if sup in  aff_dict:# 标号是不是在sups里面
                    author_aff_list.append(aff_dict[sup])
#                 elif first:
#                     author_aff_list.append(aff_dict['first'])
                if (sup=='*' and equal and (author_index>2)) or (not equal and sup=='*') : # 解决共同第一的问题
                    corresp=1
                    email = corresp_email.pop(0)
                    print(email,equal)
        elif first:#没有 sup也就是只有一个单位
            author_aff_list.append(aff_name)
            
        for author_aff_name in author_aff_list:
            cu.execute('insert into journal_author values(%s, %s, %s, %s, %s, %s)',
               (pmid, author_index, author, author_aff_name, email, corresp,))
            con.commit()
                
    #keyword
    for kw in root.cssselect('ul.keywordList li'):
#         print(kw.text_content().replace(';', '').strip())
        cu.execute('insert into journal_keyword values(%s, %s)', (pmid, kw.text_content().replace(';', '').strip(),))
        con.commit()
    

In [151]:

def springer(pmid, url):
#     response = requests.get(url, headers=headers)
    text = requests.get(url, headers = headers).content
    root = fromstring(text)
    # aff
    aff_index = 0
    for aff in root.cssselect('ul.author-affiliations li'):
        aff_index +=1
        for span in aff.findall('span'):
            if 'affiliation' in span.get('class'):
                aff_name = span.text_content().strip()
                cu.execute('insert into journal_aff (pmid, aff_index, aff_name) values(%s, %s, %s)', (pmid, aff_index, aff_name,))
                con.commit()
    # author
    author_index = 0
    for author_li in root.cssselect('ul.authors li'):
        corresp=0
        author=None
        email = None
        author_aff_name = None
        author_index +=1
        for a in author_li.findall('a'):
            if 'person' in a.get('class'):
                author = a.text_content()
            if 'envelope' in a.get('class'):
                corresp = 1
                email = a.get('title')
        author_aff_list = [aff.get('title') for aff in  author_li.findall('sup')]
        if author_aff_list:
            for author_aff_name in author_aff_list:
                cu.execute('insert into journal_author values(%s, %s, %s, %s, %s, %s)',
                           (pmid, author_index, author, author_aff_name, email, corresp,))
                con.commit()
        else:
            cu.execute('insert into journal_author values(%s, %s, %s, %s, %s, %s)',
                           (pmid, author_index, author, author_aff_name, email, corresp,))
            con.commit()
            
    # keywords
#     keywords = root.cssselect('span.Keyword')
    for kw in root.cssselect('span.Keyword'):
        cu.execute('insert into journal_keyword values(%s, %s)', (pmid, kw.text_content().strip(),))
        con.commit()     
    

In [117]:
def elsevier(pmid, url):
    response = requests.get(url)
    server = response.headers['Server']
    if server == 'www.sciencedirect.com':
        get_sciencedirect(pmid, response.content)
        
    return(response.content)

In [131]:
def get_sciencedirect(pmid, text):
    pmid = int(pmid)
    root = fromstring(text)
#     ul = root.cssselect('ul.authorGroup')[0]
    # aff
    aff_dict = {}
    aff_index = 0
    for aff in root.cssselect('ul.affiliation li'):
        aff_index +=1
        aff_id = aff.get('id')
        aff_name = aff.find('span').text_content()
        aff_dict[aff_id] = str(aff_name)
        cu.execute('insert into journal_aff (pmid, aff_index, aff_name) values(%s, %s, %s)', (pmid, aff_index, aff_name,))
        con.commit()
    # author
    author_index = 0
    for author_li in root.cssselect('ul.authorGroup li'):
        a_list = author_li.findall('a')
        corresp=0
        author=None
        email = None
        author_aff_name = None
        author_index +=1
        author_aff_list = []
        for a in a_list:
            if 'authorName' in a.get('class'):
                author = a.text_content()
            if a.get('title') and 'Corresponding author contact information' in a.get('title'):
                corresp = 1
#                 print(corresp)
            if a.get('class') and a.get('href') and 'auth_mail' in a.get('class'):
                email = a.get('href').replace('mailto:', '')
#                 print(email)
            if a.get('title') and 'Affiliation' in a.get('title') or len(aff_dict)==1:
                
                if len(aff_dict) == 1:
#                     author_aff_name=aff_dict[aff_id]
                    author_aff_list.append(aff_dict[aff_id])
                else:
                    aff = a.get('href').replace('#', '')
#                     author_aff_name = aff_dict[aff]
                    author_aff_list.append(aff_dict[aff])
        if author_aff_list:
            for author_aff_name in author_aff_list:
                cu.execute('insert into journal_author values(%s, %s, %s, %s, %s, %s)',
                           (pmid, author_index, author, author_aff_name, email, corresp,))
                con.commit()
        else:
            cu.execute('insert into journal_author values(%s, %s, %s, %s, %s, %s)',
                           (pmid, author_index, author, author_aff_name, email, corresp,))
            con.commit()

                

    #keywords
    keywords = root.cssselect('ul.keyword li')
    for kw in keywords:
        cu.execute('insert into journal_keyword values(%s, %s)', (pmid, kw.text_content().replace(';', '').strip(),))
        con.commit()
            

In [225]:
con = pymysql.connect(host='localhost', user='root', passwd='', db='xiaobaifinder', charset='utf8')
cu = con.cursor()
cu.execute('create table if not exists journal_keyword(pmid int, keywords varchar(200))')
cu.execute('create table if not exists journal_aff(pmid int,aff_index int, aff_name Text)')
cu.execute('create table if not exists journal_author(pmid int,author_index int,\
            author varchar(200), author_aff Text, email varchar(200), corresp int)')
con.commit()
pmid=1
# text = elsevier(pmid, url)
wiley(7, url)
cu.close()
con.close()






/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/pymysql/cursors.py:146: Warning: Table 'journal_keyword' already exists
  result = self._query(query)
/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/pymysql/cursors.py:146: Warning: Table 'journal_aff' already exists
  result = self._query(query)
/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/pymysql/cursors.py:146: Warning: Table 'journal_author' already exists
  result = self._query(query)
/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/ipykernel/__main__.py:37: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.


In [3]:
def get_content(url):
    try:
        storage = io.BytesIO()
        c = pycurl.Curl()
        c.setopt(c.URL, url)
        c.setopt(pycurl.FOLLOWLOCATION, 1) 
        c.setopt(pycurl.WRITEFUNCTION, storage.write)
        # c.setopt(c.HTTPHEADER,["Accept: application/vnd.crossref.unixsd+xml"])
        c.perform()
        out = c.getinfo(pycurl.EFFECTIVE_URL)
#         cu_w.execute('insert into doi_link values(%s, %s, %s)', (pmid,doi, out))
#         con.commit()
        c.close()
        content = storage.getvalue().decode('utf-8')
#         print(out)
#         print(content)
        storage.close()
        return content,out
    except Exception as e:
        print(e)

In [64]:
con = pymysql.connect(host='localhost', user='root', passwd='', db='xiaobaifinder', charset='utf8')
cu_r = con.cursor()
cu_r.execute('insert into tmp values(%s)', (None,))
con.commit()
cu_r.close()
con.close()